In [1]:
from simple.ohlc import debounce, midPrice, TTrade, npJoin
from simple.plotly import chartFigure
from simple.funcs import symlog
import numpy as np

In [2]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)[:2000]
T[-5:]

rec.array([('2022-05-08T00:01:50.167000', 1995, 2638.5 , 2.000e+00),
           ('2022-05-08T00:01:50.167000', 1996, 2638.5 , 5.275e+03),
           ('2022-05-08T00:01:50.167000', 1997, 2638.56, 5.000e+00),
           ('2022-05-08T00:01:50.167000', 1998, 2638.62, 3.704e+03),
           ('2022-05-08T00:01:50.167000', 1999, 2638.63, 3.000e+00)],
          dtype={'names':['DateTimeA','LocalTimeA','PriceA','VolumeA'], 'formats':['<M8[us]','<i8','<f8','<f8'], 'offsets':[0,8,16,24], 'itemsize':40})

In [3]:
Buy = T[T.VolumeA > 0]
Sell = T[T.VolumeA < 0]

In [4]:
C = debounce(T)
C[-5:]

rec.array([('2022-05-08T00:01:47.215000', 496, 2638.445,       0, 2.0000e+00, 2.000e+00,    0.,  1,  1,  0),
           ('2022-05-08T00:01:47.281000', 497, 2638.495, 2886000, 1.3646e+04, 1.277e+04, -876., 44, 33, 11),
           ('2022-05-08T00:01:50.167000', 498, 2638.555,       0, 5.0000e+00, 5.000e+00,    0.,  1,  1,  0),
           ('2022-05-08T00:01:50.167000', 499, 2638.615,       0, 3.7040e+03, 3.704e+03,    0.,  1,  1,  0),
           ('2022-05-08T00:01:50.167000', 500, 2638.625,       0, 3.0000e+00, 3.000e+00,    0.,  1,  1,  0)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Volume', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i4'), ('BuyCount', '<i4'), ('SellCount', '<i4')])

In [5]:
len(T), len(C)

(2000, 501)

In [6]:
# indexed chart
fig = chartFigure(height=400,
    Sell=dict(x=Sell.DateTimeA, y=Sell.PriceA, color='red', mode='markers', size=-symlog(Sell.VolumeA)+3, opacity=0.2),
    Buy=dict(x=Buy.DateTimeA, y=Buy.PriceA, color='green', mode='markers', size=symlog(Buy.VolumeA)+3, opacity=0.2),
    Debounce=dict(mode='markers+lines', x=C.DT, y=C.Price, color='black', connectgaps=True, opacity=0.7)
)
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.2,
    …

In [7]:
C.Volume = C.BuySize + C.SellSize
Buy = C[C.Volume > 0]
Sell = C[C.Volume < 0]
Sell[-5:]

rec.array([('2022-05-08T00:01:44.597000', 471, 2637.905,      0,   -77., 0.,   -77., 1, 0, 1),
           ('2022-05-08T00:01:44.743999', 481, 2638.135,      0,  -190., 0.,  -190., 1, 0, 1),
           ('2022-05-08T00:01:45.020000', 483, 2638.135, 493000, -2523., 0., -2523., 7, 0, 7),
           ('2022-05-08T00:01:46.519000', 485, 2638.135,      0,   -19., 0.,   -19., 1, 0, 1),
           ('2022-05-08T00:01:46.765000', 491, 2638.295, 438000, -1606., 4., -1610., 6, 2, 4)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Volume', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i4'), ('BuyCount', '<i4'), ('SellCount', '<i4')])

In [8]:
# indexed chart without bidask
fig = chartFigure(height=400,
    Sell=dict(x=Sell.Index, y=Sell.Price, color='red', mode='markers', size=-symlog(Sell.Volume)-1, opacity=0.4),
    Buy=dict(x=Buy.Index, y=Buy.Price, color='green', mode='markers', size=symlog(Buy.Volume)-1, opacity=0.4),
)
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.4,
    …

In [9]:
T = np.zeros(len(C), dtype=TTrade).view(np.recarray)
T.DateTimeA = C.DT
T.PriceA = C.Price#np.round(C.Price, 1)
T.VolumeA = C.Volume
T[-5:]

rec.array([('2022-05-08T00:01:47.215000', '1970-01-01T00:00:00.000000', 2638.445, 2.0000e+00, 0.),
           ('2022-05-08T00:01:47.281000', '1970-01-01T00:00:00.000000', 2638.495, 1.1894e+04, 0.),
           ('2022-05-08T00:01:50.167000', '1970-01-01T00:00:00.000000', 2638.555, 5.0000e+00, 0.),
           ('2022-05-08T00:01:50.167000', '1970-01-01T00:00:00.000000', 2638.615, 3.7040e+03, 0.),
           ('2022-05-08T00:01:50.167000', '1970-01-01T00:00:00.000000', 2638.625, 3.0000e+00, 0.)],
          dtype=[('DateTimeA', '<M8[us]'), ('LocalTimeA', '<M8[us]'), ('PriceA', '<f8'), ('VolumeA', '<f8'), ('OpenIntA', '<f8')])

In [10]:
C = debounce(T)
len(T), len(C)

(501, 501)

In [11]:
C.Volume = C.BuySize + C.SellSize
Buy = C[C.Volume > 0]
Sell = C[C.Volume < 0]
Sell[-5:]

rec.array([('2022-05-08T00:01:44.597000', 471, 2637.91, 0,   -77., 0.,   -77., 1, 0, 1),
           ('2022-05-08T00:01:44.743999', 481, 2638.14, 0,  -190., 0.,  -190., 1, 0, 1),
           ('2022-05-08T00:01:45.020000', 483, 2638.14, 0, -2523., 0., -2523., 1, 0, 1),
           ('2022-05-08T00:01:46.519000', 485, 2638.14, 0,   -19., 0.,   -19., 1, 0, 1),
           ('2022-05-08T00:01:46.765000', 491, 2638.3 , 0, -1606., 0., -1606., 1, 0, 1)],
          dtype=[('DT', '<M8[us]'), ('Index', '<i4'), ('Price', '<f8'), ('Duration', '<m8[us]'), ('Volume', '<f8'), ('BuySize', '<f8'), ('SellSize', '<f8'), ('Count', '<i4'), ('BuyCount', '<i4'), ('SellCount', '<i4')])

In [12]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S[:5]

rec.array([('2022-05-08T00:01:00.007000', 2636., 2635.99),
           ('2022-05-08T00:01:00.043000', 2636., 2635.99),
           ('2022-05-08T00:01:00.079000', 2636., 2635.99),
           ('2022-05-08T00:01:00.117000', 2636., 2635.99),
           ('2022-05-08T00:01:00.149000', 2636., 2635.99)],
          dtype={'names':['DT','A','B'], 'formats':['<M8[us]','<f4','<f4'], 'offsets':[0,8,16], 'itemsize':24})

In [13]:
K = npJoin(C.DT, S.DT)
K

array([   9,   16,   16,   16,   16,   16,   16,   16,   21,   21,   21,
         24,   66,   67,   73,   73,   73,   74,   74,   74,   74,   82,
         98,  104,  162,  163,  163,  163,  180,  180,  180,  200,  200,
        200,  200,  200,  200,  200,  200,  200,  200,  200,  200,  200,
        201,  201,  201,  201,  201,  201,  201,  201,  201,  201,  201,
        201,  201,  201,  201,  202,  203,  203,  203,  205,  205,  205,
        205,  205,  205,  205,  205,  208,  210,  214,  215,  215,  216,
        216,  216,  217,  217,  217,  218,  218,  218,  219,  219,  219,
        219,  219,  219,  219,  220,  220,  220,  221,  222,  222,  223,
        223,  223,  224,  224,  224,  224,  224,  224,  225,  225,  225,
        225,  225,  226,  226,  226,  226,  227,  227,  228,  228,  229,
        229,  229,  229,  229,  230,  230,  230,  230,  230,  231,  234,
        235,  241,  241,  241,  241,  241,  241,  346,  346,  346,  346,
        346,  346,  346,  346,  346,  346,  362,  3

In [14]:
len(C), len(S), len(K)

(501, 99941, 501)

In [17]:
# indexed chart without bidask
fig = chartFigure(height=600, rows=2,
    Sell=dict(x=Sell.Index, y=Sell.Price, color='red', mode='markers', size=-symlog(Sell.Volume)-1, opacity=0.75),
    Buy=dict(x=Buy.Index, y=Buy.Price, color='green', mode='markers', size=symlog(Buy.Volume)-1, opacity=0.75),
    Bid=dict(x=C.Index, y=S.B[K], color='green', opacity=0.4),
    Ask=dict(x=C.Index, y=S.A[K], color='red', opacity=0.4)
)
fig.add_bar(x=Buy.Index, y=Buy.Volume, row=2, col=1, marker_color='green')
fig.add_bar(x=Sell.Index, y=Sell.Volume, row=2, col=1, marker_color='red')
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.75,
   …